# Bayesian Neural Networks for Amplitude Regression - Student Version

## Introduction

This tutorial introduces **Bayesian Neural Networks (BNNs)** applied to amplitude regression in particle physics. Unlike traditional neural networks that provide point estimates, Bayesian neural networks capture uncertainty in both predictions and model parameters.

### What You'll Learn
- How to implement variational inference in neural networks
- The reparameterization trick for gradient-based optimization
- KL divergence regularization in Bayesian models
- Practical applications to particle physics amplitude regression

### Key Concepts

**Bayesian Neural Networks**: Instead of learning deterministic weights, BNNs learn distributions over weights, enabling uncertainty quantification.

**Variational Inference**: An approximate method to perform Bayesian inference by optimizing a variational distribution to approximate the true posterior.

**Reparameterization Trick**: A technique to make stochastic nodes differentiable by expressing random variables as deterministic functions of noise.

### Prerequisites
- Basic knowledge of neural networks
- Familiarity with PyTorch
- Understanding of probability distributions

Let's start by importing the necessary libraries and setting up our environment.

## Understanding the Data

We'll work with synthetic amplitude data that simulates particle physics experiments. The goal is to predict amplitudes based on input features while quantifying our uncertainty about these predictions. 
We want to train a network to predict the amplitude for the process $gg \rightarrow \gamma \gamma g$. So the amplitude depends on the 4-momentum of 5 particles: 2 incoming gluons, 2 outgoing photons, and one outgoing gluon. The incoming gluons will have no transverse momentum, but their total momentum along the beam pipe is not necessarily zero.

The dataset contains:
- **Input features**: 5 particles' 4-momenta (20 features)
- **Target amplitudes**: Real-valued outputs representing physical amplitudes

This is an ideal scenario for Bayesian methods since we want to understand not just what the amplitude is, but how confident we are in our prediction.

### Imports

In [2]:
import os
import sys
import random
import time
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import Tensor
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import functional as F
from torch.nn import init
from torch.nn import Module

#### Plotting set-up

In [3]:
from matplotlib import pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings("ignore")
from matplotlib.lines import Line2D
from matplotlib.font_manager import FontProperties
import matplotlib.colors as mcolors
import colorsys

## Loading the data

Set the path to the get_data.py file.

In [ ]:
# Download data first if needed
!python3 data/get_data.py 1 data

Trying to download dataset 1 ('amplitude regression', used in tutorials [2, 3, 4]) from https://www.thphys.uni-heidelberg.de/~plehn/data/tutorial-2-data.zip to tutorial-2-data.zip
100%|███████████████████████████████████████████| 11/11 [00:00<00:00, 39.73it/s]
Successfully extracted files from tutorial-2-data.zip
get_data.py             tutorial-2-data (1).zip tutorial-2-data (3).zip
tutorial-2-data         tutorial-2-data (2).zip tutorial-2-data.zip


In [7]:
trn_dat = np.load("data/tutorial-2-data/trn_dat.npy")
trn_amp = np.load("data/tutorial-2-data/trn_amp.npy")

val_dat = np.load("data/tutorial-2-data/val_dat.npy")
val_amp = np.load("data/tutorial-2-data/val_amp.npy")

tst_dat = np.load("data/tutorial-2-data/tst_dat.npy")
tst_amp = np.load("data/tutorial-2-data/tst_amp.npy")

In [8]:
print(f"train data shape: {trn_dat.shape}")
print(f"train amp  shape: {trn_amp.shape}")
print(f"test  data shape: {tst_dat.shape}")
print(f"test  amp  shape: {tst_amp.shape}")
print(f"val   data shape: {val_dat.shape}")
print(f"val   amp  shape: {val_amp.shape}")

train data shape: (30000, 5, 4)
train amp  shape: (30000,)
test  data shape: (30000, 5, 4)
test  amp  shape: (30000,)
val   data shape: (30000, 5, 4)
val   amp  shape: (30000,)


## Preprocessing the data

In [9]:
def get_pt(fv):
    """ returns p_T of given four vector """
    ptsq = np.round(fv[:, 1]**2 + fv[:, 2]**2, 5)
    return np.sqrt(ptsq)

In [10]:
# Preprocessing steps (similar to previous tutorials)
trn_dat_gluon_pt = get_pt(trn_dat[:, 4])

nev = trn_dat.shape[0]
trn_datf = np.reshape(trn_dat, (nev,-1))
val_datf = np.reshape(val_dat, (nev,-1))
tst_datf = np.reshape(tst_dat, (nev,-1))

gpt = np.mean(trn_dat_gluon_pt)
trn_datf = trn_datf / gpt
val_datf = val_datf / gpt
tst_datf = tst_datf / gpt

trn_datfp = torch.Tensor(trn_datf)
val_datfp = torch.Tensor(val_datf)
tst_datfp = torch.Tensor(tst_datf)

trn_ampl = np.log(trn_amp)
val_ampl = np.log(val_amp)
tst_ampl = np.log(tst_amp)

trn_amplp = torch.Tensor(trn_ampl)
val_amplp = torch.Tensor(val_ampl)
tst_amplp = torch.Tensor(tst_ampl)

## Datasets and dataloaders

In [11]:
class amp_dataset(Dataset):
    
    def __init__(self, data, amp):
        self.data = data
        self.amp = amp

    def __len__(self):
        return len(self.amp)

    def __getitem__(self, idx):
        return self.data[idx], self.amp[idx]

In [12]:
trn_dataset = amp_dataset(trn_datfp, trn_amplp.unsqueeze(-1))
val_dataset = amp_dataset(val_datfp, val_amplp.unsqueeze(-1))
tst_dataset = amp_dataset(tst_datfp, tst_amplp.unsqueeze(-1))

trn_dataloader = DataLoader(trn_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
tst_dataloader = DataLoader(tst_dataset, batch_size=64, shuffle=True)

## The Variational Bayesian Linear Layer

The core of our Bayesian neural network is the **VBLinear** layer. Unlike standard linear layers that have deterministic weights, this layer treats weights as probability distributions.

### Key Components:
1. **Mean parameters** (μ): The central tendency of weight distributions
2. **Rho parameters** (ρ): Related to the variance via σ = log(1 + exp(ρ))
3. **Reparameterization**: w = μ + σ ⊙ ε, where ε ~ N(0,1)
4. **KL divergence**: Regularization term measuring distance from prior

First let's look at the source code for a **basic linear layer** in pytorch:

(https://pytorch.org/docs/stable/_modules/torch/nn/modules/linear.html#Linear)

```
class Linear(Module):
    
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input: Tensor) -> Tensor:
        return F.linear(input, self.weight, self.bias)

    def extra_repr(self) -> str:
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )
```

Objects of this class apply a linear transformation to the incoming data: $y = xA^T + b$.

The input arguments are required to initialise the layer, so, in the \_\_init()\_\_ function.  We have:
- in\_features: size of each input sample
- out\_features: size of each output sample
- bias: If set to ``False``, the layer will not learn an additive bias.  Default: ``True``

The shapes are:
- Input: $(*, H_{in})$ where $*$ means any number of dimensions including none and $H_{in} = \text{in\_features}$.
- Output: $(*, H_{out})$ where all but the last dimension are the same shape as the input and $H_{out} = \text{out\_features}$.

The layer has attributes:
- weight: the learnable weights of the module of shape $(\text{out\_features}, \text{in\_features})$. The values are initialized from $\mathcal{U}(-\sqrt{k}, \sqrt{k})$, where $k = \frac{1}{\text{in\_features}}$
- bias:   the learnable bias of the module of shape $(\text{out\_features})$.  If `bias` is ``True``, the values are initialized from $\mathcal{U}(-\sqrt{k}, \sqrt{k})$ where $k = \frac{1}{\text{in\_features}}$.

Examples::

    >>> m = nn.Linear(20, 30)
    
    >>> input = torch.randn(128, 20)
    
    >>> output = m(input)
    
    >>> print(output.size())
    
    torch.Size([128, 30])

From the lecture, we know that in a Bayesian network the weights are replaced by Gaussian distributions, and on a forward pass we get the output by sampling from that distribution.

So the biases are the same as in the linear layer.  But not each weight is a Gaussian distibution and so needs a mean and a variance.  The bias and the mean and variance of the weight distribution will be learnable.  In practice it's easier to work with the log of the variance as it is more stable when optimising the network.

We need to be able to sample from the Gaussian weight distributions, and compute derivatives of the output in order to update the network parameters.  To do this we use something called the 're-parameterisation trick'.  It involves sampling random noise from a unit normal distribution, and then transforming that number using the mean and variance of the weight distribution in this way:
\begin{equation}
w = \mu + \sigma\times r
\end{equation}
where $r$ is a random number sampled from a unit normal distribution (Gaussian with mean and variance equal to one), $\mu$ is the mean of the weight distribution, and $\sigma$ is the standard deviation.  In this way we separate the stochastic part of the function from the parameters defining the distribution.  And so if we take any differentiable function of $x$ (e.g. an activation function), we can compute derivatives of that function with respect to the mean and variance of the weight distribution.

In the `forward` method we then need to implement this reparameterisation trick for the weights, and then apply the same linear transformation as in the standard linear layer.

On each forward pass we need to generate a set of random numbers with the same shape as our means and variances.  Choosing a set of random numbers for the sampling is equivalent to 'sampling' a new neural network from the Bayesian neural network.  And sometimes at the end of the analysis, we will want to keep the same network for testing.  So we don't always want to re-sample the random numbers on each forward pass.  To control this we define a flag called `self.resample`, with default set to ``True``.

We also need a `reset_parameters` function to reset the parameters in the network.  This is standard for layers in pytorch.  We use a slightly different function to do this than is used in the pytorch linear layer, as can be seen below.

From the lecture, you know that the weight distributions require a prior.  The simplest choice for this prior is just a Gaussian distribution with a mean and variance of one.  Results are typically not too sensitive to this prior, as long as the values are within reasonable limits.  For example, $\mathcal{O}(1)$ means and standard deviations.  Going beyond $\mathcal{O}(1)$ numbers just leads to numerical instabilities in the training.  The Bayesian loss function contains a term coming from the KL divergence between the weight distributions in the network and their priors.  So the layers should have some funcitonality to return these values.  The KL divergence for the layer is:
\begin{equation}
\text{KL} = \sum_{\text{weights}} 0.5 \times (  \mu^2 + \sigma^2 - \log\sigma^2 - 1 )
\end{equation}

So let's build the **simplest Bayesian layer** we can.

## Exercise 1: Implement the VBLinear Layer

Your task is to complete the implementation of the Variational Bayesian Linear layer. This layer is the building block of Bayesian neural networks.

**Instructions:**
1. Complete the `__init__` method to initialize the required parameters
2. Implement the `forward` method using the reparameterization trick
3. Implement the `reset_parameters` method for proper initialization
4. Complete the `KL` method to compute KL divergence

**Hints:**
- Use `Parameter(Tensor(...))` for learnable parameters
- The reparameterization trick: `weight = μ + σ * ε` where `ε ~ N(0,1)`
- KL divergence formula: `0.5 * (μ² + σ² - log(σ²) - 1)`

In [13]:
class VBLinear(Module):
    # VB -> Variational Bayes
    
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor
    
    def __init__(self, in_features, out_features):
        super(VBLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.resample = True
        
        # TODO: Initialize the parameters
        # You need:
        # - bias: Parameter for bias terms
        # - mu_w: Parameter for weight means
        # - logsig2_w: Parameter for log variance of weights
        # - random: tensor for storing random noise (not a parameter)
        
        # YOUR CODE HERE
        pass
        
        self.reset_parameters()
        
    def forward(self, inpt):
        # TODO: Implement the forward pass
        # 1. Generate new random numbers if self.resample is True
        # 2. Compute the variance from log variance
        # 3. Apply reparameterization trick to get weights
        # 4. Apply linear transformation
        
        # YOUR CODE HERE
        pass
    
    def reset_parameters(self):
        # TODO: Initialize parameters
        # Initialize mu_w with normal distribution
        # Initialize logsig2_w with small values around -9
        # Initialize bias to zero
        
        # YOUR CODE HERE
        pass
        
    def KL(self, loguniform=False):
        # TODO: Compute KL divergence
        # Formula: 0.5 * sum(μ² + σ² - log(σ²) - 1)
        
        # YOUR CODE HERE
        pass

## Exercise 2: Test the VBLinear Layer

Test your implementation by creating two VBLinear layers and passing data through them.

In [14]:
# Test your VBLinear implementation
tlr0 = VBLinear(10, 5)
tlr1 = VBLinear(5, 2)

# Test data
x = torch.rand(20, 10)

# Forward pass
output = tlr1(tlr0(x))
print(f"Output shape: {output.shape}")

# Test stochasticity - run multiple times
print("\nTesting stochasticity (should be different each time):")
for i in range(5):
    print(tlr0(x)[0][0])

AttributeError: 'NoneType' object has no attribute 'shape'

## Exercise 3: Implement the Bayesian Loss Function

The Bayesian loss function combines two components:
1. Negative log Gaussian likelihood
2. KL divergence from the network weights

Implement the negative log Gaussian function:

In [ ]:
def neg_log_gauss(outputs, targets):
    # TODO: Implement negative log Gaussian
    # outputs[:, 0] contains the predicted means
    # outputs[:, 1] contains the log variance predictions
    # Formula: (μ - y)² / (2σ²) + 0.5 * log(σ²)
    
    # YOUR CODE HERE
    pass

## Exercise 4: Build the Bayesian Neural Network

Complete the implementation of the Bayesian neural network class:

In [ ]:
ipt_dim = 20
opt_dim = 1
hdn_dim = 50

class bayes_amp_net(Module):
    
    def __init__(self, training_size, hdn_dim=50):
        super(bayes_amp_net, self).__init__()
        
        self.training_size = training_size
        self.vb_layers = []
        self.all_layers = []

        # TODO: Build the network architecture
        # 1. Input layer: VBLinear(ipt_dim, hdn_dim) + ReLU
        # 2. Two hidden layers: VBLinear(hdn_dim, hdn_dim) + ReLU each
        # 3. Output layer: VBLinear(hdn_dim, 2) for mean and log variance
        
        # YOUR CODE HERE
        pass

        self.model = nn.Sequential(*self.all_layers)

    def forward(self, x):
        # TODO: Implement forward pass
        # YOUR CODE HERE
        pass

    def KL(self):
        # TODO: Compute total KL divergence from all VB layers
        # Don't forget to normalize by training_size
        # YOUR CODE HERE
        pass
    
    def neg_log_gauss(self, outputs, targets):
        # TODO: Use the neg_log_gauss function you implemented
        # YOUR CODE HERE
        pass

## Exercise 5: Training Loop

Implement the training function for the Bayesian neural network:

In [ ]:
def train_epoch(dataloader, model, optimizer):
    model.train()
    loss_tot = 0.0
    
    for batch, (X, y) in enumerate(dataloader):
        # TODO: Implement training step
        # 1. Forward pass through model
        # 2. Compute negative log Gaussian loss
        # 3. Compute KL divergence
        # 4. Total loss = neg_log_gauss + KL
        # 5. Backpropagation and optimizer step
        
        # YOUR CODE HERE
        pass
        
    return loss_tot / len(dataloader)

def validate_epoch(dataloader, model):
    model.eval()
    loss_tot = 0.0

    with torch.no_grad():
        for X, y in dataloader:
            #TODO: Implement validation step
            pass

    return loss_tot / len(dataloader)

## Exercise 6: Training and Evaluation

Set up the training loop and train your Bayesian neural network:

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Initialize model
trn_len = trn_amplp.shape[0]
model = bayes_amp_net(trn_len, hdn_dim=50).to(device)
batch_size = 128

trn_dataset = amp_dataset(trn_datfp, trn_amplp.unsqueeze(-1)).to(device)
val_dataset = amp_dataset(val_datfp, val_amplp.unsqueeze(-1)).to(device)
tst_dataset = amp_dataset(tst_datfp, tst_amplp.unsqueeze(-1)).to(device)

trn_dataloader = DataLoader(trn_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
tst_dataloader = DataLoader(tst_dataset, batch_size=batch_size, shuffle=False)

# TODO: Set up optimizer and training parameters
# YOUR CODE HERE

# TODO: Implement training loop
# YOUR CODE HERE

## Exercise 7: Uncertainty Quantification

Implement functions to extract predictions and uncertainties from your trained Bayesian model:

In [ ]:
def get_prediction(model, dataloader, n_monte=30):
    # TODO: Implement Monte Carlo sampling
    # 1. Run the model multiple times (n_monte) to get different samples
    # 2. Collect predictions for both mean and variance
    # 3. Return samples for further analysis
    
    # YOUR CODE HERE
    pass

## Exercise 8: Analysis and Visualization

Create visualizations to analyze your results:
1. Plot training/validation loss curves
2. Compare predicted vs true amplitudes
3. Analyze uncertainty estimates
4. Compute and plot pull distributions

In [ ]:
# TODO: Implement analysis and visualization
# YOUR CODE HERE